In [2]:
# Import the modules
from sklearn.externals import joblib
from sklearn import datasets
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import RandomizedPCA
import numpy as np
from collections import Counter
from sklearn.datasets import load_digits
import csv
import random
import math
import operator
import matplotlib.pyplot as plt
from termcolor import colored,cprint

# Import datasets, classifiers and performance metrics

from sklearn.datasets import load_iris
from sklearn import datasets, svm, metrics
from sklearn.cross_validation import KFold
import cv2


# K Nearest Neighbours Implementation

In [3]:
#####################  Euclidean Distance   #########################

def euclideanDistance(x1, x2, length):
    distance = 0
    for x in range(length):
        distance += pow((x1[x] - x2[x]), 2)
    return math.sqrt(distance)

In [4]:
#####################  FInd the Neighbors   #########################


def Find_Neighbors(trainingSet, testData, k):
    distances = []
    
    length = len(testData)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testData, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [5]:
######################### Predict the data   ########################

def Predict(neighbors):
    class_votes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in class_votes:
            class_votes[response] += 1
        else:
            class_votes[response] = 1
    sorted_votes = sorted(class_votes.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sorted_votes[0][0]

In [6]:
#########################  Find Accuracy of My model     ########################


def Find_Accuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        
        if testSet[x] == predictions[x]:
            correct += 1
            print colored(">>Predicted Result {0} and Actual Result {1}" \
                          .format(predictions[x],testSet[x]),'green')
    
        else:
            #pass
            cprint(">>Predicted Result {0} and Actual Result {1}" \
                          .format(predictions[x],testSet[x]),None,'on_red')

    return ((correct/float(len(testSet))) * 100.0),correct

#  Implementation of Principal Component Analysis

In [8]:
#############################  Implementation of PCA  ##########################

def Find_PCA(train, trainlab, test, labels, num=None):

    if num is None:
        num=len(test)

    train_pca = np.array(train)
    test_pca = np.array(test)

    n_components = 10

    pca = RandomizedPCA(n_components=n_components).fit(train_pca)
    X_Train = pca.transform(train_pca)
    Y_Train = trainlab
    X_Test = pca.transform(test_pca)
    Y_Test = labels
    y_predictions = []
    y_test = []
    
    
    #print X_Train.shape,Y_Train.shape
    #print X_Test.shape,Y_Test.shape
    trainingSet = np.concatenate((X_Train, Y_Train), axis=1)
    testSet = np.concatenate((X_Test, Y_Test), axis=1)
    predictions=[]
    k = 3

    for x in range(len(testSet)):
        neighbors = Find_Neighbors(trainingSet, testSet[x], k)

        result = Predict(neighbors)
        #print int(result)
        y_predictions.append((result))
        y_test.append((testSet[x][-1]))
        #print('>> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
    
    #print y_predictions,y_test
    accuracy,correct = Find_Accuracy(y_test, y_predictions)

    print "\n\nTested ", num, " digits"
    print "correct: ", correct, "wrong: ", (num-correct), "error rate: ", \
                float(num-correct)*100/(correct+(num-correct)), "%"
    print "got correctly ", float(correct)*100/(correct+(num-correct)), "%"
    
    return y_predictions

# MNIST Handwritten Digit Training Part

In [9]:
##############################  MNIST Digit Training Part for HOG Features  #####################################

# Hog Fetures are used for Object Detection. So as our project is about Handwritten digit recognition, we have 
#     used Hog feature.  


# digits = load_digits()
# np.unique(digits.target)

# features = np.array(digits.data, 'int16') 
# labels = np.array(digits.target, 'int')
# #print features[0].shape

# list_hog_fd = []
# for feature in features:
#     fd = hog(feature.reshape((8, 8)), orientations = 9, pixels_per_cell=(4, 4), \
#              cells_per_block=(1, 1), visualise=False)
#     list_hog_fd.append(fd)
# hog_features = np.array(list_hog_fd, 'float64')



dataset = datasets.fetch_mldata("MNIST Original")

# Extract the features and labels
features = np.array(dataset.data, 'int16') 
labels = np.array(dataset.target, 'int')

# print features[0].shape
# Extract the hog features
list_hog_fd = []
for feature in features:
    # Hog Feature for object Identification
    fd = hog(feature.reshape((28, 28)), orientations=9, pixels_per_cell=(14, 14), \
             cells_per_block=(1, 1), visualise=False)
    list_hog_fd.append(fd)
hog_features = np.array(list_hog_fd, 'float64')

# Count the number of Samples for every digit
print "Count of digits in dataset", Counter(labels)
accuracy = 0.0

X,Y = hog_features,labels
Y = Y.reshape(Y.shape[0],1)

Count of digits in dataset Counter({1: 7877, 7: 7293, 3: 7141, 2: 6990, 9: 6958, 0: 6903, 6: 6876, 8: 6825, 4: 6824, 5: 6313})


In [10]:
X_train, X_test = X[:60000], X[69900:]
Y_train, Y_test = Y[:60000], Y[69900:]


print Y_train.shape,X_test.shape

(60000, 1) (100, 36)


# Apply the Principal Component Analysis algorithm for KNN. 

In [10]:
y_predicted = Find_PCA(X_train, Y_train, X_test, Y_test)

>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 7.0 and Actual Result 9.0
>>Predicted Result 8.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicted Result 9.0 and Actual Result 9.0
>>Predicte

# 1st Handwritten Digit Testing Image

In [55]:
####################################  Harsh Handwritting Testing part    #################################### 

# Read the input image 

im = cv2.imread("harsh.png")

# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (3, 3), 0)

############## Original Image  ###################

cv2.imshow("Original Image",im)

# Threshold the image
ret, im_th = cv2.threshold(im_gray, 150, 255, cv2.THRESH_BINARY_INV)

# Find contours in the image
#cv2.imshow("Threshold",im_th)
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

testingSet = [cv2.boundingRect(ctr) for ctr in ctrs]

trainingSet = np.concatenate((X_train, Y_train), axis=1)

print "Wait for a moment......"

print "\n\nMy Predicted Result \n"
i = 0

for rect in testingSet:
    
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    
    roi = cv2.dilate(roi, (4, 4))
    
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    #print (roi_hog_fd).shape
    k = 3
    
    neighbors = Find_Neighbors(trainingSet, roi_hog_fd, k)
    
    result = Predict(neighbors)
    #print result
    y_test = np.array([[[5]],[[2]],[[7]]])
    y_predicted = Find_PCA(X_train, Y_train, roi_hog_fd, y_test[i])
    i+=1
    #print y_predicted
    cv2.putText(im, str(int(result)), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
    
cv2.imshow("Resulting Image", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Wait for a moment......


My Predicted Result 

5.0
2.0
7.0


# Plot the Accuracy Curve

In [5]:
##########################  Plot the Accuracy Curve  ###############################


plt.plot([5,10,20,40,60,80,100,200,400,784], [77.3,80.2,81.2,81.9,82.7,81.2,82.8,81.7,82.9,86.0],'-ro')
plt.xlabel("Number of Features")
plt.ylabel("Accuracy")
plt.axis([0, 800, 0, 100])
plt.text(400,90,"Inbuilt Accuracy = 87.0 %")
plt.title("KNN Accuracy Varies with Features ")
plt.show()

# KNN and PCA Implemetation using Inbuilt Method

In [11]:
#########################  Inbuilt Method for KNN and PCA   ##########################

# http://brianfarris.me/static/digit_recognizer.html

from sklearn.decomposition import RandomizedPCA
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

x_train, x_test = X[:60000], X[69900:]
y_train, y_test = Y[:60000], Y[69900:]


pca = RandomizedPCA(n_components=10).fit(x_train)


xtrain = pca.transform(x_train)

xtest = pca.transform(x_test)

clf = KNeighborsClassifier()

#fitting knn
clf = clf.fit(xtrain, y_train)

#predicting
y_pred = clf.predict(xtest)

acc_knn = accuracy_score(y_test, y_pred)

print "K-Nearest Neighbors with PCA Accuracy: ",(acc_knn*100) , "%"

K-Nearest Neighbors with PCA Accuracy:  87.0 %


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
